In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
model_path = "/home/ubuntu/Dimitris/results/second_v3_phase_xcpetion_model.h5"

In [3]:
import os
import numpy as np
import pickle
from keras.models import load_model
from keras import backend as K

Using TensorFlow backend.


In [4]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.

    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.

    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.

    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    """Computes the f-measure, the harmonic mean of precision and recall.

    Here it is only computed as a batch-wise average, not globally.
    """
    return fbeta_score(y_true, y_pred, beta=1)

In [5]:
model = load_model(model_path,custom_objects={'fmeasure': fmeasure})

In [6]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input

In [7]:
def chunks(chunkable, n):
    for i in range(0, len(chunkable), n):
        yield chunkable[i:i+n]

In [8]:
images_path = "/home/ubuntu/test/images/jpeg/CaptionTesting2018/"
names_list  = os.listdir(images_path)
batch_size  = 200

In [9]:
def images_generator(img_size = (224, 224, 3), suffix = ''):
    for sub_list in chunks(names_list, batch_size):
        imgs = np.zeros((len(sub_list), *img_size))
        for i, name in enumerate(sub_list):
            filename = images_path + name + suffix
            img = load_img(filename, target_size=img_size[:2], interpolation="bicubic")
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = preprocess_input(img, mode='tf')
            imgs[i] = img
        yield imgs

In [10]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [11]:
resultspath = "/home/ubuntu/test/results/"
thresholdpath = "/home/ubuntu/Dimitris/results/"

In [12]:
for i, images in enumerate(images_generator()):
    result = model.predict_on_batch(images)
    save_obj(result, resultspath + "result_" + str(i).format(3))

In [13]:
result_0 = load_obj(resultspath + "result_0")

# AUTO THA PREPEI NA GFINEI UPDATE

In [14]:
best_threshold = load_obj(thresholdpath + "best_threshold")

In [15]:
labelspath = "/home/ubuntu/Dimitris/"
labels_list = load_obj(labelspath + "labels_list")

In [16]:
indexes_list = []
for result_path in os.listdir(resultspath):
    result_data = load_obj(resultspath + result_path.replace('.pkl', ''))
    for image_result in result_data:
        labels = image_result > best_threshold
        indexes = np.where(labels)[0]
        indexes_list.append(indexes)

In [17]:
with open("/home/ubuntu/Dimitris/outputs.txt", 'w') as f:
    for name, indexes in zip(names_list, indexes_list):
        line = name.replace('.jpg', '') + "\t" + ','.join([labels_list[index] for index in indexes]) + '\n'
        f.write(line)